Currently active strategies:
- RAR: strategy 1 (+1620)
- KLP: strategy 3, "round" variation (+562)
- SQI: ??? (probably some sort of pattern identification)

RAR Strategy 1: maximum amount of 9998 bids and 10002 asks every timestamp. Historical result: +1620

In [6]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string

class Trader:
    '''
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

		# Orders to be placed on exchange matching engine
        result = {}
        for product in state.order_depths:
            order_depth: OrderDepth = state.order_depths[product]
            orders: List[Order] = []
            acceptable_price = 10  # Participant should calculate this value
            print("Acceptable price : " + str(acceptable_price))
            print("Buy Order depth : " + str(len(order_depth.buy_orders)) + ", Sell order depth : " + str(len(order_depth.sell_orders)))
    
            if len(order_depth.sell_orders) != 0:
                best_ask, best_ask_amount = list(order_depth.sell_orders.items())[0]
                if int(best_ask) < acceptable_price:
                    print("BUY", str(-best_ask_amount) + "x", best_ask)
                    orders.append(Order(product, best_ask, -best_ask_amount))
    
            if len(order_depth.buy_orders) != 0:
                best_bid, best_bid_amount = list(order_depth.buy_orders.items())[0]
                if int(best_bid) > acceptable_price:
                    print("SELL", str(best_bid_amount) + "x", best_bid)
                    orders.append(Order(product, best_bid, -best_bid_amount))
            
            result[product] = orders
    
		    # String value holding Trader state data required. 
				# It will be delivered as TradingState.traderData on next execution.
        traderData = "SAMPLE" 
        
				# Sample conversion request. Check more details below. 
        conversions = 1
        return result, conversions, traderData
    '''
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        # maximum orders to not exceed the limit
        pos = 0
        if 'RAINFOREST_RESIN' in state.position:
            pos = state.position['RAINFOREST_RESIN']

        
            
        result['RAINFOREST_RESIN'] = [Order('RAINFOREST_RESIN', 9998, 50-pos), Order('RAINFOREST_RESIN', 10002, -50-pos)]
        traderData = ""
        conversions = 0
        return result, conversions, traderData

RAR Strategy 2: artificially drive the price up and down. (not implemented yet)

In [ ]:
# not implemented yet
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        traderData = ""
        conversions = 0
        return result, conversions, traderData

RAR Strategy 3: only pick up whatever is profitable in the market. Result: +892 (why so low? too passive?)

In [13]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))
        
        result = {}
        orders: List[Order] = []

        RAR = 'RAINFOREST_RESIN'  # alias

        # strategy for RAR
        stable_price = 10000
        
        # maximum orders to not exceed the limit
        pos = 0
        if RAR in state.position:
            pos = state.position[RAR]

        max_buy_orders = 50 - pos
        max_sell_orders = 50 + pos
        
        # optimally place buy orders
        order_depth: OrderDepth = state.order_depths[RAR]
        
        for bid_price, bid_volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if (bid_price <= stable_price):
                continue
            if (max_sell_orders > 0):
                my_order = min(max_sell_orders, bid_volume)
                orders.append(Order(RAR, bid_price, -my_order))  # ask
                max_sell_orders -= my_order
            
        for ask_price, ask_volume in sorted(order_depth.sell_orders.items()):
            if (ask_price >= stable_price):
                continue
            if (max_buy_orders > 0):
                my_order = min(max_buy_orders, -ask_volume)
                orders.append(Order(RAR, ask_price, my_order))  # bid
                max_buy_orders -= my_order
        
        result[RAR] = orders

        print('sending out orders', orders)
        
        traderData = ""
        conversions = 0
        return result, conversions, traderData

RAR Strategy 4: Combine strategy 1 (MM) and strategy 3 (pick up scraps). Result is the same: +1620.
(btw, if MM gap is +- 1 only, the profit reduces to +1254.)

In [ ]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))
        
        result = {}
        orders: List[Order] = []

        RAR = 'RAINFOREST_RESIN'  # alias

        # strategy for RAR
        stable_price = 10000
        
        # maximum orders to not exceed the limit
        pos = 0
        if RAR in state.position:
            pos = state.position[RAR]

        max_buy_orders = 50 - pos
        max_sell_orders = 50 + pos
        
        # optimally place buy orders
        order_depth: OrderDepth = state.order_depths[RAR]
        
        for bid_price, bid_volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if (bid_price <= stable_price):
                continue
            if (max_sell_orders > 0):
                my_order = min(max_sell_orders, bid_volume)
                orders.append(Order(RAR, bid_price, -my_order))  # ask
                max_sell_orders -= my_order
            
        for ask_price, ask_volume in sorted(order_depth.sell_orders.items()):
            if (ask_price >= stable_price):
                continue
            if (max_buy_orders > 0):
                my_order = min(max_buy_orders, -ask_volume)
                orders.append(Order(RAR, ask_price, my_order))  # bid
                max_buy_orders -= my_order

        # combine with MM orders
        orders.append(Order(RAR, 9998, max_buy_orders))  # bid
        orders.append(Order(RAR, 10002, -max_sell_orders))  # ask
        result[RAR] = orders

        print('sending out orders', orders)
        
        traderData = ""
        conversions = 0
        return result, conversions, traderData

KLP strategy 1: make the bid and ask using floor(average bid) + 1 and ceil(average ask) - 1. Result: +554.

In [3]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import math

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        LIMIT = 50

        # strategy for KLP
        
        KLP = 'KELP'
        order_depth: OrderDepth = state.order_depths[KLP]

        # maximum orders to not exceed the limit
        KLP_pos = 0
        if KLP in state.position:
            KLP_pos = state.position[KLP]

        # compute bid price as just above the weighted average of bid price
        total_bid_volume = 0
        total_bid_prices = 0
        
        for bid_price, bid_volume in order_depth.buy_orders.items():
            total_bid_volume += bid_volume
            total_bid_prices += bid_price * bid_volume

        # what if total_bid_volume == 0?
        my_bid_price = math.floor(total_bid_prices / total_bid_volume) + 1
        
        total_ask_volume = 0
        total_ask_prices = 0
        
        for ask_price, ask_volume in order_depth.sell_orders.items():
            total_ask_volume += (-ask_volume)
            total_ask_prices += ask_price * (-ask_volume)

        my_ask_price = math.ceil(total_ask_prices / total_ask_volume) - 1

        if (total_bid_volume > 0 and total_ask_volume > 0 and my_bid_price < my_ask_price):
            # only then, make bids and asks
            result[KLP] = [Order(KLP, my_bid_price, LIMIT - KLP_pos), Order(KLP, my_ask_price, -LIMIT - KLP_pos)]

        print('all orders:', result)
        
        traderData = ""
        conversions = 0
        return result, conversions, traderData

KLP strategy 2: make the bid and ask using the current best market bids. Result: -54

In [4]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import math

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        LIMIT = 50

        # strategy for KLP
        
        KLP = 'KELP'
        order_depth: OrderDepth = state.order_depths[KLP]

        # maximum orders to not exceed the limit
        KLP_pos = 0
        if KLP in state.position:
            KLP_pos = state.position[KLP]

        # compute bid price as just above the weighted average of bid price
        my_bid_price = 0
        
        for bid_price, bid_volume in order_depth.buy_orders.items():
            my_bid_price = max(my_bid_price, bid_price)
        
        my_ask_price = 9999
        for ask_price, ask_volume in order_depth.sell_orders.items():
            my_ask_price = min(my_ask_price, ask_price)

        if (my_bid_price < my_ask_price):
            result[KLP] = [Order(KLP, my_bid_price, LIMIT - KLP_pos), Order(KLP, my_ask_price, -LIMIT - KLP_pos)]

        print('all orders:', result)
        
        traderData = ""
        conversions = 0
        return result, conversions, traderData

KLP strategy 3: instead of B+1 and A-1, use 0.75A + 0.25B and 0.25A + 0.75B. Result: +470. Using round, +562.

In [ ]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import math

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        LIMIT = 50

        # strategy for KLP
        
        KLP = 'KELP'
        order_depth: OrderDepth = state.order_depths[KLP]

        # maximum orders to not exceed the limit
        KLP_pos = 0
        if KLP in state.position:
            KLP_pos = state.position[KLP]

        # compute bid price as just above the weighted average of bid price
        total_bid_volume = 0
        total_bid_prices = 0
        
        for bid_price, bid_volume in order_depth.buy_orders.items():
            total_bid_volume += bid_volume
            total_bid_prices += bid_price * bid_volume

        # what if total_bid_volume == 0?
        avg_bid_price = total_bid_prices / total_bid_volume
        
        total_ask_volume = 0
        total_ask_prices = 0
        
        for ask_price, ask_volume in order_depth.sell_orders.items():
            total_ask_volume += (-ask_volume)
            total_ask_prices += ask_price * (-ask_volume)

        avg_ask_price = total_ask_prices / total_ask_volume

        my_bid_price = round(0.75 * avg_bid_price + 0.25 * avg_ask_price)
        my_ask_price = round(0.25 * avg_bid_price + 0.75 * avg_ask_price)

        if (total_bid_volume > 0 and total_ask_volume > 0 and my_bid_price < my_ask_price):
            # only then, make bids and asks
            result[KLP] = [Order(KLP, my_bid_price, LIMIT - KLP_pos), Order(KLP, my_ask_price, -LIMIT - KLP_pos)]

        print('all orders:', result)
        
        traderData = ""
        conversions = 0
        return result, conversions, traderData

SQI strategy 1: basic mean reversion. (need to be careful when optimizing the parameters.)

In [9]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import math
import jsonpickle

class Trader:
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

        traderData = jsonpickle.decode(state.traderData)
        
        # initialize
        SQI_history = []
        if (isinstance(traderData, dict)):
            SQI_history = traderData['SQI_history']

        result = {}
        orders: List[Order] = []

        LIMIT = 50
        SQI = 'SQUID_INK'
            
        # strategy for SQI
        order_depth_SQI: OrderDepth = state.order_depths[SQI]
        SQI_pos = 0
        if SQI in state.position:
            SQI_pos = state.position[SQI]

        # see if is bull / is bear
        # if bull + see immediate sell signal, sell!
        # if bear + see immediate buy signal, buy!

        # MA200 > MA1000 = bull
        # MA200 < MA1000 = bear
        # each time tick only market buy/sell 1 unit?

        # price > MA50 * 1.02 = spike up
        # price < MA50 * 0.98 = spike down
        # immediately switch
        
        # compute the mid price
        SQI_bid_volume = 0
        SQI_bid_prices = 0
        for bid_price, bid_volume in order_depth_SQI.buy_orders.items():
            SQI_bid_volume += bid_volume
            SQI_bid_prices += bid_price * bid_volume

        SQI_avg_bid = SQI_bid_prices / SQI_bid_volume

        SQI_ask_volume = 0
        SQI_ask_prices = 0
        for ask_price, ask_volume in order_depth_SQI.sell_orders.items():
            SQI_ask_volume += (-ask_volume)
            SQI_ask_prices += ask_price * (-ask_volume)

        SQI_avg_ask = SQI_ask_prices / SQI_ask_volume

        SQI_mid_price = (SQI_avg_bid + SQI_avg_ask) / 2

        SQI_history.append(SQI_mid_price)



        is_bull = False
        is_bear = False
        spike_up = False
        spike_down = False
        MA50 = -1
        MA200 = -1
        MA1000 = -1
        
        T = len(SQI_history)
        if (T >= 50):
            MA50 = sum(SQI_history[-50:]) / 50
        if (T >= 200):
            MA200 = sum(SQI_history[-200:]) / 200
        if (T >= 1000):
            MA1000 = sum(SQI_history[-1000:]) / 1000

        if (T >= 1000):
            is_bull = (MA200 > MA1000)
            is_bear = (MA200 < MA1000)
        if (T >= 50):
            spike_up = (SQI_mid_price >= 1.02 * MA50)
            spike_down = (SQI_mid_price <= 0.98 * MA50)

        my_buy = 0
        my_sell = 0
        max_buy = LIMIT - SQI_pos
        max_sell = LIMIT + SQI_pos
        # just do market orders?
        buy_price = round(SQI_avg_ask)
        sell_price = round(SQI_avg_bid)
        if (spike_up):
            my_sell = max_sell
            sell_price -= 1
        elif (spike_down):
            my_buy = max_buy
            buy_price += 1
        elif (is_bull):
            my_buy = min(max_buy, 1)  # only buy 1
        elif (is_bear):
            my_sell = min(max_sell, 1) # only sell 1

        SQI_orders = []
        if (my_buy > 0):
            SQI_orders.append(Order(SQI, buy_price, my_buy))
        if (my_sell > 0):
            SQI_orders.append(Order(SQI, sell_price, -my_sell))
        result[SQI] = SQI_orders
        
        print('all orders:', result)
        
        traderData = jsonpickle.encode({'SQI_history': SQI_history})
        conversions = 0
        return result, conversions, traderData

Here is an aggregation of the current best strategies:

In [ ]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import math
import jsonpickle

class Trader:
    def run(self, state: TradingState):
       # print("traderData: " + state.traderData)
       # print("Observations: " + str(state.observations))

        result = {}
        orders: List[Order] = []

        LIMIT = 50

        RAR = 'RAINFOREST_RESIN'
        KLP = 'KELP'
        SQI = 'SQUID_INK'


        
        # strategy for RAR
        
        pos_RAR = 0
        if RAR in state.position:
            pos_RAR = state.position[RAR]

        my_bid_RAR = 9998
        my_ask_RAR = 10002

        result[RAR] = [Order(RAR, 9998, LIMIT - pos_RAR), Order(RAR, 10002, -LIMIT - pos_RAR)]



        
        # strategy for KLP
        
        order_depth_KLP: OrderDepth = state.order_depths[KLP]

        # maximum orders to not exceed the limit
        pos_KLP = 0
        if KLP in state.position:
            pos_KLP = state.position[KLP]

        # compute bid price as just above the weighted average of bid price
        total_bid_volume = 0
        total_bid_prices = 0
        
        for bid_price, bid_volume in order_depth_KLP.buy_orders.items():
            total_bid_volume += bid_volume
            total_bid_prices += bid_price * bid_volume

        # what if total_bid_volume == 0?
        avg_bid_price = total_bid_prices / total_bid_volume
        
        total_ask_volume = 0
        total_ask_prices = 0
        
        for ask_price, ask_volume in order_depth_KLP.sell_orders.items():
            total_ask_volume += (-ask_volume)
            total_ask_prices += ask_price * (-ask_volume)

        avg_ask_price = total_ask_prices / total_ask_volume

        my_bid_KLP = round(0.75 * avg_bid_price + 0.25 * avg_ask_price)
        my_ask_KLP = round(0.25 * avg_bid_price + 0.75 * avg_ask_price)

        if (total_bid_volume > 0 and total_ask_volume > 0 and my_bid_KLP < my_ask_KLP):
            # only then, make bids and asks
            result[KLP] = [Order(KLP, my_bid_KLP, LIMIT - pos_KLP), Order(KLP, my_ask_KLP, -LIMIT - pos_KLP)]






        
        SQI_history = []
        if (state.traderData != ""):
            SQI_history = jsonpickle.decode(state.traderData)['SQI_history']
            
        # strategy for SQI
        order_depth_SQI: OrderDepth = state.order_depths[SQI]
        SQI_pos = 0
        if SQI in state.position:
            SQI_pos = state.position[SQI]

        # see if is bull / is bear
        # if bull + see immediate sell signal, sell!
        # if bear + see immediate buy signal, buy!

        # MA200 > MA1000 = bull
        # MA200 < MA1000 = bear
        # each time tick only market buy/sell 1 unit?

        # price > MA50 * 1.02 = spike up
        # price < MA50 * 0.98 = spike down
        # immediately switch
        
        # compute the mid price
        SQI_bid_volume = 0
        SQI_bid_prices = 0
        for bid_price, bid_volume in order_depth_SQI.buy_orders.items():
            SQI_bid_volume += bid_volume
            SQI_bid_prices += bid_price * bid_volume

        SQI_avg_bid = SQI_bid_prices / SQI_bid_volume

        SQI_ask_volume = 0
        SQI_ask_prices = 0
        for ask_price, ask_volume in order_depth_SQI.sell_orders.items():
            SQI_ask_volume += (-ask_volume)
            SQI_ask_prices += ask_price * (-ask_volume)

        SQI_avg_ask = SQI_ask_prices / SQI_ask_volume

        SQI_mid_price = (SQI_avg_bid + SQI_avg_ask) / 2

        SQI_history.append(SQI_mid_price)



        is_bull = False
        is_bear = False
        spike_up = False
        spike_down = False
        MA50 = -1
        MA200 = -1
        MA1000 = -1
        
        T = len(SQI_history)
        if (T >= 50):
            MA50 = sum(SQI_history[-50:]) / 50
        if (T >= 200):
            MA200 = sum(SQI_history[-200:]) / 200
        if (T >= 1000):
            MA1000 = sum(SQI_history[-1000:]) / 1000

        if (T >= 1000):
            is_bull = (MA200 > MA1000)
            is_bear = (MA200 < MA1000)
        if (T >= 50):
            spike_up = (SQI_mid_price >= 1.02 * MA50)
            spike_down = (SQI_mid_price <= 0.98 * MA50)

        SQI_buy = 0
        SQI_sell = 0
        max_SQI_buy = LIMIT - SQI_pos
        max_SQI_sell = LIMIT + SQI_pos
        # just do market orders?
        SQI_buy_price = round(SQI_avg_ask)
        SQI_sell_price = round(SQI_avg_bid)
        if (spike_up):
            SQI_sell = max_SQI_sell
            SQI_sell_price -= 1
        elif (spike_down):
            SQI_buy = max_SQI_buy
            SQI_buy_price += 1
        elif (is_bull):
            SQI_buy = min(max_SQI_buy, 1)  # only buy 1
        elif (is_bear):
            SQI_sell = min(max_SQI_sell, 1) # only sell 1

        SQI_orders = []
        if (SQI_buy > 0):
            SQI_orders.append(Order(SQI, SQI_buy_price, SQI_buy))
        if (SQI_sell > 0):
            SQI_orders.append(Order(SQI, SQI_sell_price, -SQI_sell))
        result[SQI] = SQI_orders





        
        print('all orders:', result)
        
        traderData = jsonpickle.encode({'SQI_history': SQI_history})
        
        conversions = 0
        return result, conversions, traderData